### real_data2 / reverse2_df 상관계수

In [1]:
# 라이브러리

import pandas as pd
import numpy as np

In [ ]:
# 데이터 불러오기

# reverse2_df.csv
df = pd.read_csv('C:/sas_view/use_code/Data_Preprocessing/data/reverse2_df.csv', index_col = 0) 
# real_data2.csv
df2 = pd.read_csv('C:/sas_view/data/real_data2.csv', index_col = 0) 

# reverse2_df [Unnamed: 0.1] 컬럼 제거
df = df.drop(columns = ['Unnamed: 0.1'])

In [ ]:
df # reverse2_df

In [ ]:
df2 # real_data2

In [ ]:
dis = [] # 질병 리스트
var = [] # 변수 리스트
Cor = [] # 상관계수 리스트

for i in range(1, df2.shape[1]) : # shape : 행열의 개수 추출 / df2.shape[1] : 열의 수 (29)
    for j in range(1, df.shape[1]) : # df.shape[1] (552)
        try :
            corr = df2.iloc[:,i].corr(df.iloc[:,j])
            if ((corr >= .4) | (corr <= -.4)) & (corr != 1.000) : # 상관계수가 0.4 이상인 데이터 추출
                dis.append(df2.columns[i])
                var.append(df.columns[j])
                Cor.append(round(corr, 3))
                
        except :
            pass

In [ ]:
# 상관계수 df

cor_df = pd.DataFrame({'dis':dis,  # {'변수명' : 데이터를 가지고 있는 리스트}
                       'var':var,
                       'Cor':Cor})
cor_df

In [ ]:
# 변수사전 csv 파일 불러오기

same = pd.read_csv('C:/sas_view/code/same_code_2.csv', index_col = 0)
same

In [ ]:
# same df '변수명' 대문자화

for i in range(0, len(same)) :
    same.iloc[i][0] = same.iloc[i][0].upper() 

same

In [ ]:
# cor_df 질병에 해당하는 변수의 수 ( 28개 )

dis_s = set(cor_df.iloc[ : , 0])
dis_s = list(dis_s)
print('cor_df 질병에 해당하는 변수의 수 : ', len(dis_s))

# cor_df 질병에 영향을 미치는 변수의 수 ( 197개 )
var_s = set(cor_df.iloc[:,1])
var_s = list(var_s)
print('cor_df 질병에 영향을 미치는 변수의 수 : ', len(var_s)) 

In [ ]:
# 질병과 그에 영향을 미치는 변수를 리스트에 담아주는 작업 실시

dis_list = []
var_list = []

for  i in range(0, len(same)) :
    if same.loc[i][0] in dis_s :
        dis_list.append(i)
    elif same.loc[i][0] in var_s :
        var_list.append(i)

print('질병 : ', len(dis_list), '\n' '변수 : ', len(var_list))

In [45]:
dis_df = same.iloc[dis_list] # 질병에 해당하는 사전
var_df = same.iloc[var_list] # 기본변수에 해당하는 사전

In [ ]:
# 상관계수 df에 변수설명을 merge하기위해 변수사전에 있는 변수명을 cor_df에 맞추어 변경하는 작업 실시

dis_df.rename(columns = {'변수명' : 'dis', '변수설명7' : 'dis_설명'}, inplace = True) 
var_df.rename(columns = {'변수명' : 'var', '변수설명7' : 'var_설명'}, inplace = True)

In [ ]:
dis_df

In [ ]:
var_df

In [59]:
# dis_df / var_df merge

cor_df2 = pd.merge(cor_df, dis_df.iloc[ : , [0, 1]]) # dis를 중심으로 merge
cor_df2 = pd.merge(cor_df2, var_df.iloc[ : , [0, 1]]) # var를 중심으로 merge
out_cor = cor_df2.sort_values('dis')
out_cor

,dis,var,Cor,dis_설명,var_설명
170,DC1_PR,DC1_AG,0.437,위암 현재 유병 여부,위암 진단시기
199,DC1_PR,DC3_AG,0.571,위암 현재 유병 여부,대장암 진단시기
25,DC1_PR,DC7_AG,0.693,위암 현재 유병 여부,갑상선암 진단시기
130,DC1_PR,DI5_AG,0.775,위암 현재 유병 여부,심근경색증 진단시기
353,DC1_PR,EC_STT_2,0.524,위암 현재 유병 여부,(취업자) 종사상지위_임금근로자 상세
...,...,...,...,...,...
101,DM4_PR,BE3_74,-0.479,골다공증 현재 유병 여부,일_고강도 신체활동 시간(분)
55,DM4_PR,DK9_PT,-0.516,골다공증 현재 유병 여부,C형간염 치료
59,DM4_PR,BE3_77,-0.420,골다공증 현재 유병 여부,여가_고강도 신체활동 시간(시간)
162,DM4_PR,DM4_PT,0.494,골다공증 현재 유병 여부,골다공증 치료


In [60]:
# 진단 시기(_AG) 빼기 

out_cor2 = out_cor[ ~ out_cor['var'].str.contains('_AG')]  # ' _AG'를 포함하고 있는 변수를 제외한 나머지 변수 추출
len(set(out_cor2.iloc[ : , 1]))

166

In [62]:
# csv 파일로 저장

out_cor.to_csv('C:/sas_view/use_code/Data_Preprocessing/data/out_cor.csv', encoding = 'cp949') # 진단시기 변수 포함
# out_cor2.to_csv('cor2_output.csv', encoding = 'cp949') # 진단시기 변수 빠짐